In [2]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time

In [3]:
# now we set a standard random seed so that we can reproduce results
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# now we split the available training data into training, test and cross validation

# Training data
n_training_samples = 20000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

# Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype = np.int64))

# Test
n_test_samples = 5000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [6]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        print("Shape before 1st conv: " + str(x.shape))
        x = F.relu(self.conv1(x))
        print("Shape after 1st conv: " + str(x.shape))
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        print("Shape after 1st pool: " + str(x.shape))
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        print("Shape after view: " + str(x.shape))
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        print("Shape after 1st fc: " + str(x.shape))
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        print("Shape at output: " + str(x.shape))
        return(x)

In [7]:
# DataLaoder
# Takes in a dataset and a sampler for loading
# num_workers deals with system memory and threads

def get_train_loader(batch_size):
    train_loader = data.DataLoader(train_set, batch_size=batch_size,
                                  sampler=train_sampler, num_workers=2)
    
    return train_loader

In [8]:
# Test and Validation loaders have constant batch size, so we can define them directly

test_loader = data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

In [9]:
# define the loss and opitmizer functions
# we use the CrossEntropyLoss and ADAM as Optimizer

def createLossAndOptimizer(net, learning_rate = 0.0001):
    # Loss function
    loss = nn.CrossEntropyLoss()
    
    # Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [10]:
# Define a function to train the CNN 

def trainNet(net, batch_size, number_of_epochs, learning_rate):
    # Print all the hyperparameters of the training iteration
    print("Hyperparameters: ")
    print("Batch size = ", batch_size)
    print("epochs = ", number_of_epochs)
    print("Learning Rate = ", learning_rate)
    
    # Get Training Data
    train_loader = get_train_loader(batch_size)
    number_of_batches = len(train_loader)
    
    # Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    # Keep track of time
    training_start_time = time.time()
    
    # Loop for number_of_epochs
    for epoch in range(number_of_epochs):
        running_loss = 0.0
        print_every = number_of_batches // 10
        start_time = time.time()
        total_train_loss = 0.0
        
        for i, data in enumerate(train_loader, 0):
            # Get inputs
            inputs, labels = data
            
            # Wraps them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            # Set the parameter gradients to zero
            # And make the forward pass, calculate gradient, do backprop
            optimizer.zero_grad()
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            # Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            # Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t Train loss: {:.2f} took: {:.2f}s".format(
                epoch+1, int(100* (i+1)/number_of_batches),
                running_loss / print_every,
                time.time() - start_time))
                
                # Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
                
        # At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            # Wrap tensors in variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            # Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data
        
        print("validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
    
    print("Training finished. Took: {:.2f}s".format(time.time() - training_start_time))
    

In [11]:
# Create the network and run it

cnn = SimpleCNN()
print(cnn)
train_on_gpu = torch.cuda.is_available() #will return true if gpu available
print("CUDA support: " + str(train_on_gpu))
# move tensors to GPU if CUDA is available
#if train_on_gpu:
    #cnn.cuda() # if gpu available train on gpu
trainNet(cnn, batch_size=32, number_of_epochs=50, learning_rate=0.0001)

SimpleCNN(
  (conv1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)
CUDA support: True
Hyperparameters: 
Batch size =  32
epochs =  20
Learning Rate =  0.001
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st c

Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape afte

Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape befo

Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64

Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size

...

Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size

Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])


Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape afte

Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Epoch 7, 40% 	 Train loss: 0.86 took: 4.88s
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st conv: torch.Size([32, 3, 32, 32])
Shape after 1st conv: torch.Size([32, 18, 32, 32])
Shape after 1st pool: torch.Size([32, 18, 16, 16])
Shape after view: torch.Size([32, 4608])
Shape after 1st fc: torch.Size([32, 64])
Shape at output: torch.Size([32, 10])
Shape before 1st 

KeyboardInterrupt: 